# Alteration Filing
## Follow these steps to complete the alteration filing for a corporation in lear

**Note:** Before running these steps this corporation will need to be loaded into lear by using the data_loader located in *lear/lear-db/test_data/data_loader.py*.

## Setup the environment

In [ ]:
%run ./bcr-business-setup.ipynb

## Set the business identifier
**eg. BC1234567**

In [ ]:
identifier = '<identifier>'

## Verify business info is correct

In [ ]:
business = legal_api.models.Business.find_by_identifier(identifier)
business.id, business.identifier, business.legal_name, business.legal_type

## Affiliate business to business account

### Authenticate

In [ ]:
token = AccountService.get_bearer_token()

### Get account number

In [ ]:
# uncomment for account name or account id

# bcol_account_id=<int>
# url = f'orgs?bcolAccountId={bcol_account_id}'

# account_name='<name>'
# url = f'orgs?name={account_name.replace(" ", "%20")}'

account_call = requests.get(
    f'{os.getenv("AUTH_URL")}/{url}',
    headers={'Content-Type': 'application/json', 'Authorization': f'Bearer {token}'}
)
account_call.status_code, account_call.json()


In [ ]:
# double check that ['orgs'][0]['id'] is the right id -> may need to change this
account_number = account_call.json()['orgs'][0]['id']
account_number

### Affiliate using the account number

In [ ]:

affiliation = AccountService.create_affiliation(
    account=account_number,
    business_registration=business.identifier,
    business_name=business.legal_name,
    corp_type_code=Business.LegalTypes.BCOMP.value
)
affiliation

## Fill out the filing json for the alteration
*note: uncomment code as needed*

### Base filing json for alteration

In [ ]:
filing_json = {
    'filing': {
        'header': {
            'name': 'alteration',
            'certifiedBy': '<name_of_person_filing>',
            'date': '<yyyy-mm-dd>',  # today
            'effectiveDate': '<datetime>',  # 1970-01-01T00:00:00+00:00

            # 'bcolAccountNumber': '<string>',          # optional - depends on payment
            # 'datNumber': '<string>',                  # optional - depends on payment
            # 'routingSlipNumber': '<string>',          # optional - depends on payment
            # 'folioNumber': '<string>'                 # optional
        },
        'business': {
            'identifier': business.identifier,
            'legalName': business.legal_name,
            'legalType': business.legal_type
        },
        'alteration': {
            'provisionsRemoved': <bool>,
            'business': {
                'identifier': business.identifier,
                # need to uncomment one of below
                # 'legalType': Business.LegalTypes.BCOMP.value
                # 'legalType': Business.LegalTypes.COMP.value
            },
            'contactPoint': {
                'email': '<string>',
                # 'phone': '<string>'  # optional
            }
        }
    }
}
filing_json

### Add on name request if required

#### Get nr info

In [ ]:
nr = NameXService.query_nr_number('<nr_num_string>')  # NR 1234567
nr.status_code, nr.json()

#### Fill in nr info

In [ ]:
name_request = {
    'legalName': '<new_name>',
    'nrNumber': '<nr_number>',
    'legalType': Business.LegalTypes.BCOMP.value
}
# filing_json['filing']['alteration']['nameRequest'] = name_request
filing_json

### Add on name translations if required

In [ ]:
# new_name_translations = [<new_translation1>, <etc>]
# modified_translations = [
#     {
#         'oldValue': <old_translation>,
#         'newValue': <new_translation>
#     }
# ]
# ceased_translations = [<existing_translation1>, <etc>]

name_translations = {
    # 'new': new_name_translations,
    # 'modified': modified_translations,
    # 'ceased': ceased_translations
}

# filing_json['filing']['alteration']['nameTranslations'] = name_translations
filing_json

### Add on alter share structure if required

#### Make base json

In [ ]:
share_structure = {
    'resolutionDates': ['<yyyy-mm-dd>', <etc.>],
    'shareClasses': []  # keep this empty
}

#### Create share classes and series json

In [ ]:
new_share_class = {
    'name': '<class_name>',
    'priority': <int>,
    'hasMaximumShares': <bool>,
    'maxNumberOfShares': <int or null>,
    'hasParValue': <bool>,
    'parValue': <float>,
    'hasRightsOrRestrictions': <bool>,
    'currency': '<string (CAD)>',
    'series': []  # keep this empty
}

*repeat below for each series in the above class*

In [ ]:
new_share_series = {
    'name': <class_name>,
    'priority': <int>,
    'hasMaximumShares': <bool>,
    'maxNumberOfShares': <int or null>,
    'hasRightsOrRestrictions': <bool>
}
new_share_class['series'].append(new_share_series)

*do not execute until all needed series were added to the new class*

In [ ]:
share_structure['shareClasses'].append(new_share_class)

*only execute after all classes were added (repeat above steps as necessary)* 

In [ ]:
# filing_json['filing']['alteration']['shareStructure'] = share_structure
filing_json

## Save the alteration filing to lear

### Post to legal_api

In [ ]:
r = requests.post(
    f'{os.getenv("LEGAL_URL")}/{business.identifier}/filings',
    json=filing_json,
    headers={'Content-Type': 'application/json', 'Authorization': f'Bearer {token}'}
)

## Verify filing was successful

In [ ]:
r.status_code, r.json()

In [ ]:
alteration = Filing.get_filings_by_type(business.id, 'alteration')
alteration.filing_date, alteration.effective_date, alteration.business_id, alteration.id, alteration.status, alteration.source, alteration.filing_json